In [ ]:
!pip install llama-index llama-index-llms-openai
%pip install llama-index-llms-ollama
%pip install llama-index-embeddings-ollama

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./data/sales1.csv")
df.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [3]:
df['quarter'] = pd.PeriodIndex(df.Date, freq='Q')
df['quarter'].head()

0    2019Q1
1    2019Q1
2    2019Q1
3    2019Q1
4    2019Q1
Name: quarter, dtype: period[Q-DEC]

In [4]:
pivot = df.pivot_table(index='quarter', values= ['Total','gross income','Quantity','cogs'], fill_value=0, aggfunc=["sum"])
pivot

sum                                    
        Quantity       Total       cogs gross income
quarter                                             
2019Q1      5510  322966.749  307587.38    15379.369

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.openai import OpenAI
import os
from dotenv import load_dotenv

In [6]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3.2:latest", request_timeout=180.0)

response = llm.complete("What is the capital of France?")
print(response)

The capital of France is Paris.


In [7]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama3.2:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
print(np.array(pass_embedding).shape, pass_embedding[1][:5],"...")

query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
print(np.array(query_embedding).shape, query_embedding[:5],"...")

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

(2, 3072) [-1.5460655689239502, -0.3834989070892334, 4.105189800262451, -1.3084664344787598, 0.7981865406036377] ...
(3072,) [-0.19883333146572113, 1.817630648612976, -0.7466331720352173, -0.6094018220901489, 1.2579606771469116] ...


In [8]:
# Load documents from the sales data directory
documents = SimpleDirectoryReader(input_dir='./data').load_data()
documents

[Document(id_='cce3f3af-431a-452e-85d9-1620165e9f31', embedding=None, metadata={'file_path': 'C:\\Development\\workspace\\python\\chatbot\\data\\sales1.csv', 'file_name': 'sales1.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 130528, 'creation_date': '2025-04-08', 'last_modified_date': '2025-04-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='750-67-8428, A, Yangon, Member, Female, Health and beauty, 74.69, 7, 26.1415, 548.9715, 1/5/2019, 13:08, Ewallet, 522.83, 4.761904762, 26.1415, 9.1\n226-31-3081, C, Naypyitaw, Normal, Female, Electronic accessories, 15.28, 5, 3.82, 80.22, 3/8/2019, 10:29, Cash, 76.4, 4.761904762, 3.8

In [28]:
# llm = Ollama(model="llama3.2:latest", request_timeout=60.0)
llm = Ollama(model="llama3.2:latest", request_timeout=180.0)
llm

Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x00000288A3826A10>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x00000288FEC1C860>, completion_to_prompt=<function default_completion_to_prompt at 0x00000288FF267CE0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.2:latest', temperature=0.75, context_window=3900, request_timeout=180.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None)

In [18]:
# Create an index from the documents
#embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
embedding = OllamaEmbedding(
    model_name="llama3.2:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

# index = VectorStoreIndex.from_documents(documents) # required OpenAI
index = VectorStoreIndex.from_documents(documents,embed_model=embedding)

# a document summary index needs both an llm and embed model for the constructor
# index = DocumentSummaryIndex.from_documents(documents, embed_model=embed_model, llm=llm)

index

In [23]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embedding
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900
Settings

_Settings(_llm=Ollama(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002888A15EC90>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x00000288FEC1C860>, completion_to_prompt=<function default_completion_to_prompt at 0x00000288FF267CE0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, base_url='http://localhost:11434', model='llama3.2:latest', temperature=0.75, context_window=3900, request_timeout=60.0, prompt_key='prompt', json_mode=False, additional_kwargs={}, is_function_calling_model=True, keep_alive=None), _embed_model=OllamaEmbedding(model_name='llama3.2:latest', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002888A15EC90>, num_workers=None, base_url='http://localhost:11434', ollama_additional_kwargs={'mirostat': 0}), _callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x000002888A1

In [29]:
# Create a query engine
# query_engine = index.as_query_engine(service_context=index)
query_engine = index.as_query_engine(llm=llm)
query_engine

In [30]:
# Define your query
query = "What were the total sales in the last quarter?" # NOT WOKRED # However, without explicit total sales data provided in the context snippet, it's challenging to calculate this. We would need all sales figures for these specific dates or a way to determine which ones qualify as part of 'last quarter' sales.",
query

'What were the total sales in the last quarter?'

In [31]:
# Execute the query
response = query_engine.query(query)
response

Response(response="The total sales figures are mostly not provided in the given data snippet, but some of them do have dates that fall within a specific range. \n\nBased on the date ranges for each transaction (e.g., '3/5/2019', '2/2/2019', etc.), we can identify the last quarter as approximately January to March 2019.\n\nHowever, without explicit total sales data provided in the context snippet, it's challenging to calculate this. We would need all sales figures for these specific dates or a way to determine which ones qualify as part of 'last quarter' sales.", source_nodes=[NodeWithScore(node=TextNode(id_='acc9ffb3-f8d2-4150-808f-a0a743d52790', embedding=None, metadata={'file_path': 'C:\\Development\\workspace\\python\\chatbot\\data\\sales1.csv', 'file_name': 'sales1.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 130528, 'creation_date': '2025-04-08', 'last_modified_date': '2025-04-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date'

In [32]:
# Execute the query
response = query_engine.query("What were the total sales?")
response.response
# wrong result : The total sales seems to be calculated by multiplying the transaction amount with a conversion factor (4.761904762). \n\n
# First, let's calculate the total transaction amount:\n \n
# $141.9 + $302.7 + ... + $252.15 = $2,335.11\n\n
# Now, we multiply this amount with the conversion factor to get:\n\n$2,335.11 * 4.761904762 ≈ $11,100.00", so

Response(response="The total sales seems to be calculated by multiplying the transaction amount with a conversion factor (4.761904762). \n\nFirst, let's calculate the total transaction amount:\n \n$141.9 + $302.7 + ... + $252.15 = $2,335.11\n\nNow, we multiply this amount with the conversion factor to get:\n\n$2,335.11 * 4.761904762 ≈ $11,100.00", source_nodes=[NodeWithScore(node=TextNode(id_='6d80a91d-0a5d-4d71-aed1-12456e932373', embedding=None, metadata={'file_path': 'C:\\Development\\workspace\\python\\chatbot\\data\\sales1.csv', 'file_name': 'sales1.csv', 'file_type': 'application/vnd.ms-excel', 'file_size': 130528, 'creation_date': '2025-04-08', 'last_modified_date': '2025-04-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1

In [91]:
# mistral:latest
model = "mistral:latest"
llm = Ollama(model=model, request_timeout=360.0)
embed_model = OllamaEmbedding( model_name=model,base_url="http://localhost:11434", )
index = VectorStoreIndex.from_documents(documents,embed_model=embedding)
Settings.llm = llm
Settings.embed_model = embed_model
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("What were the total sales?")
response.response
# !!! ReadTimeout: timed out

ReadTimeout: timed out

In [ ]:
# mistral:latest
model = "mistral:latest"
llm = Ollama(model=model, request_timeout=360.0)
embed_model = OllamaEmbedding( model_name=model,base_url="http://localhost:11434", )
index = VectorStoreIndex.from_documents(documents,embed_model=embedding)
Settings.llm = llm
Settings.embed_model = embed_model
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("What were the total sales?")
response.response
# !!! ReadTimeout: timed out

In [ ]:
model = "mistral:latest"
llm = Ollama(model=model, request_timeout=180.0)
embed_model = OllamaEmbedding( model_name=model,base_url="http://localhost:11434", )
documents = SimpleDirectoryReader(input_dir='./data').load_data()
#index = DocumentSummaryIndex.from_documents(documents, embed_model=embed_model, llm=llm)
index = VectorStoreIndex.from_documents(documents,embed_model=embed_model)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900
query_engine = index.as_query_engine(llm=llm)
#response = query_engine.query("What were the total sales?")
response = query_engine.query("What were the total sales in the last quarter?")
response.response